<img src="logo.png" width="10%">

In [1]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, HTML
from IPython.display import display
from pyscal_rdf.structure import StructureGraph

import gui.plot
import gui.create
from gui.create_element import CreateElement
from gui.create_structure import CreateStructure
from gui.read_structure import ReadStructure
from gui.write_structure import WriteStructure
from gui.sparql_query import SparqlQuery
from gui.auto_query import AutoQuery
from gui.ontology import Ontology
from gui.properties import dataprops

In [2]:
#Create a general element viewer
atom_viewer = gui.create.output()

In [3]:
#Element based creation routines
el = CreateElement(theme="teal")

def create_element(b=None):
    element = el.dropdown.value
    add_to_graph = el.checkbox.value
    rep = el.repetition_box.value
    global sys
    sys = g.create_element(element, add_to_graph=add_to_graph, 
                          repetitions=[rep, rep, rep]) 
    
    gui.plot.system(sys, atom_viewer)
    create_count_text()
    
@el.run_button.on_click
def element_plot_on_click(b):
    sys = create_element()

In [4]:
#Structure creation widgets
st = CreateStructure(theme="teal")

def create_structure(b=None):
    """
    Function to create structure
    """
    structure = st.dropdown.value
    add_to_graph = st.checkbox.value
    rep = st.repetition_box.value
    lattice_parameter = st.lattice_parameter_box.value
    element = st.element_box.value
    global sys
    sys = g.create_structure(structure, 
                             lattice_constant=lattice_parameter,
                             repetitions=[rep, rep, rep],
                             add_to_graph=add_to_graph,
                             element=element)
    
    gui.plot.system(sys, atom_viewer)
    create_count_text()
    return sys
    
@st.run_button.on_click
def structure_plot_on_click(b):
    sys = create_structure()

In [5]:
#Read structure widgets
rs = ReadStructure(theme="teal")

def read_structure(b=None):
    #write file
    with open("temp", "wb") as fout:
        fout.write(rs.upload.value[0]['content'])
    global sys
    sys = g.read_structure("temp", 
                           format=rs.dropdown.value)
    
    gui.plot.system(sys, atom_viewer)
    create_count_text()
    return sys    
    
@rs.run_button.on_click
def process_file_on_click(b):
    sys = read_structure()

In [6]:
#write structure widgets
ws = WriteStructure(theme="teal")

def write_structure(b=None):
    sys.to_file(f'out.{ws.dropdown.value}', format=ws.dropdown.value)

@ws.run_button.on_click
def write_file_on_click(b):
    write_structure()
    ws.output.clear_output()
    with ws.output:
        display(gui.create.download(f'out.{ws.dropdown.value}'))

In [7]:
#Creation panel
creation_panel = VBox(children=[el.panel,
                      st.panel,
                      rs.panel])


vis_label = gui.create.header("Visualise structure", theme="teal")
vis_structure_panel = VBox(children=[vis_label,
                      atom_viewer,
                      ws.panel,
                      ])

creation_tab = HBox(children=[creation_panel, 
                              vis_structure_panel])

In [8]:
#Widgets for querying
sq = SparqlQuery()

@sq.run_button.on_click
def query_plot_on_click(b):
    run_query()
    
def run_query(b=None):
    query = sq.query_box.value
    res = g.graph.query(query)
    sq.output.clear_output()
    with sq.output:
        for r in res:
            print(r[0])

In [9]:
#Ontology visualisation widgets
on = Ontology()
on.output.clear_output()
with on.output:
    display(IFrame(src="https://service.tib.eu/webvowl/#iri=http://purls.helmholtz-metadaten.de/cmso/", width='100%', height='500px'))

In [10]:
#Automated query code
aq = AutoQuery()

def update_compare_output(change):
    aq.compare_output.clear_output()
    if aq.property_dropdown.value in dataprops.keys():
        with aq.compare_output:
            display(aq.compare_options)

with aq.compare_output:
    display(aq.compare_options)
    
aq.property_dropdown.observe(update_compare_output)

def update_input_fields(change):
    aq.input_output.clear_output()
    if aq.compare_options.value == " equal to ":
        equal_inp = HBox(children=[aq.input_field_1])
        with aq.input_output:
            display(equal_inp)
    elif aq.compare_options.value == " in between ":
        equal_inp = HBox(children=[aq.input_field_1, 
                                   gui.create.text("and"), 
                                   aq.input_field_2])
        with aq.input_output:
            display(equal_inp)
    elif aq.compare_options.value == " equal to either ":
        equal_inp = HBox(children=[aq.input_field_1, 
                                   gui.create.text("or"), 
                                   aq.input_field_2])
        with aq.input_output:
            display(equal_inp)

with aq.input_output:
    display(HBox(children=[aq.input_field_1]))
    
aq.compare_options.observe(update_input_fields)

def convert_to_datatype(value, dtype):
    if dtype == "string":
        return str(value)
    elif dtype == "integer":
        return int(value)
    else:
        return float(value)
    
@aq.run_button.on_click
def run_query_on_click(b):
    run_auto_query()
    
def run_auto_query(b=None):
    #if it is equal to
    dtype = dataprops[aq.property_dropdown.value]["dtype"]
    if aq.compare_options.value == " equal to ":
        value = convert_to_datatype(aq.input_field_1.value, dtype)
    elif aq.compare_options.value == " in between ":
        value = [convert_to_datatype(aq.input_field_1.value, dtype),
                 convert_to_datatype(aq.input_field_2.value, dtype)]
    else:
        value = [convert_to_datatype(aq.input_field_1.value, "string"),
                 convert_to_datatype(aq.input_field_2.value, "string")]
    
    results = g.query_sample(dataprops[aq.property_dropdown.value]["onto"],
              value, 
              return_query=False)
    results = [r[0] for r in results]
    global result_dropdown
    result_dropdown = gui.create.dropdown("Samples", results)
    disp = HBox(children=[result_dropdown,
                         aq.plot_structure_button,
                         aq.visualise_graph_button])
    aq.result_output.clear_output()
    with aq.result_output:
        display(disp)



@aq.plot_structure_button.on_click
def plot_structure_on_click(b):
    run_plot_structure()

def run_plot_structure(b=None):
    #get sys from sample
    sys = g.get_system_from_sample(result_dropdown.value) 
    gui.plot.system(sys, aq.result_plotter)
    
@aq.visualise_graph_button.on_click
def visualise_graph_on_click(b):
    run_visualise_graph()

def run_visualise_graph(b=None):
    #get sys from sample
    sgraph, na = g.get_sample(result_dropdown.value, 
                              no_atoms=True)
    aq.result_plotter.clear_output()
    dot = sgraph.visualise(backend="graphviz")
    with aq.result_plotter:
        display(dot)
  

In [11]:
#Create a structure graph
g = StructureGraph()

In [12]:
#widget to show number of samples
#this has to come after the main graph is created
count_output = gui.create.output()

def create_count_text():
    text = "Database has %d samples"%g.n_samples
    onto_label = gui.create.text(text)
    count_output.clear_output()
    with count_output:
        display(onto_label)

create_count_text()    

In [13]:
#final tab arrangement
tab = widgets.Tab(children=[creation_tab, 
                            sq.panel,
                            aq.panel,
                            on.panel])
tab.set_title(0, 'Structure Creation')
tab.set_title(1, 'Run SPARQL Query')
tab.set_title(2, 'Run Automated Query')
tab.set_title(3, 'Ontology')


main = widgets.VBox(children=[tab,
                             count_output])
main